All code from this cell is within notebook number 5

In [1]:
import os
import cv2
import numpy as np
import tqdm as tqdm
from tqdm import tqdm as tqdm

REBUILD_DATA = False

class DogsVsCats():
    IMG_SIZE = 50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    LABELS = {CATS: 0, DOGS: 1}     # 'classes', cat is 0, dog is 1
    catcount = 0                    # a count to test the balance of the training set
    dogcount = 0
    training_data = []              # will be populated with cats,dogs and corresponding labels for every cat and dog  
    
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for file in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, file)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)    # is colour a defining feature of wether something is a cat or dog? no 
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))    # resize
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  # [img, one hot vector]

                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1

                except Exception as e:
                    pass
                    # print(str(e))
                    
        # shuffle
        np.random.shuffle(self.training_data)
        # save np.save(save as, save what)
        np.save("training_data.npy", self.training_data)
            
        print('Cats: ', self.catcount)
        print('Dogs: ', self.dogcount)
            
            
            
if REBUILD_DATA:
    dogsvcats = DogsVsCats()
    dogsvcats.make_training_data()

training_data = np.load('training_data.npy', allow_pickle=True)
print(len(training_data))

24946


Torch Time 🔥

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # 
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        # 
        x = torch.randn(50,50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
       
        
        self.fc1 = nn.Linear(self._to_linear, 512) # here we flatten 
        self.fc2 = nn.Linear(512, 2)     # output is same number as number of classes, 2
        
        # in order to get the size of the input sample for the first Linear after the last Conv2d,
        # we'll pass through dummy data in order to be able to caluculate the size.
        # We calculate the size by multiplying the dimensions of the output from the last Conv2d
        
    # This function serves as part of our forward method, however 
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))

        print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2] # np.prod(x[0].shape)
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear) # here we flatten 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)  # distribution
        

net = Net()

torch.Size([128, 2, 2])
